In [2]:
import numpy as np
import pygalmesh
import SimpleITK as sitk
from nanomesh.utils import show_slice, show_volume, generate_mesh_from_binary_image

class Cross_A(pygalmesh.DomainBase):
    def __init__(self):
        super().__init__()
        
        self.npts = 11
        self.x = np.linspace(0,1,self.npts)
        self.data = np.zeros((self.npts,self.npts,self.npts))
        idx = (self.x>0.25) & (self.x<0.75)
        all_idx = list(range(self.npts))
        self.data[np.ix_(idx,idx,all_idx)] = 1.
        self.data[np.ix_(idx,all_idx,idx)] = 1.
        self.data[np.ix_(all_idx,idx,idx)] = 1.
                    

    def eval(self, x):
        ix = np.argmin(np.abs(x[0]-self.x))
        iy = np.argmin(np.abs(x[1]-self.x))
        iz = np.argmin(np.abs(x[2]-self.x))
        if self.data[ix,iy,iz] == 1:
            return -1.
        else:
            return 1.
        
class Cross_B(pygalmesh.DomainBase):
    def __init__(self):
        super().__init__()
        
        self.npts = 11
        self.x = np.linspace(0,1,self.npts)
        self.data = np.ones((self.npts,self.npts,self.npts))
        idx = (self.x>0.25) & (self.x<0.75)
        all_idx = list(range(self.npts))
        self.data[np.ix_(idx,idx,all_idx)] = 0.
        self.data[np.ix_(idx,all_idx,idx)] = 0.
        self.data[np.ix_(all_idx,idx,idx)] = 0.
                    

    def eval(self, x):
        ix = np.argmin(np.abs(x[0]-self.x))
        iy = np.argmin(np.abs(x[1]-self.x))
        iz = np.argmin(np.abs(x[2]-self.x))
        if self.data[ix,iy,iz] == 1:
            return -1.
        else:
            return 1.
        
class FullCube(pygalmesh.DomainBase):
    def __init__(self):
        super().__init__()
        
        self.npts = 11
        self.x = np.linspace(0,1,self.npts)
        self.data = np.ones((self.npts,self.npts,self.npts))
#         idx = (self.x>0.05) & (self.x<0.95)
#         all_idx = list(range(self.npts))
#         self.data[np.ix_(idx,idx,idx)] = 1.
#         self.data[np.ix_(idx,idx,idx)] = 1.
#         self.data[np.ix_(idx,idx,idx)] = 1.
        
    def eval(self, x):
        ix = np.argmin(np.abs(x[0]-self.x))
        iy = np.argmin(np.abs(x[1]-self.x))
        iz = np.argmin(np.abs(x[2]-self.x))
        if self.data[ix,iy,iz] == 1:
            return -1.
        else:
            return 1.


In [3]:
mesh = pygalmesh.generate_periodic_mesh_multiple_domains(
    [Cross_A(),FullCube()],
    ["--","+-"],
    [0, 0, 0, 1, 1, 1],
    max_cell_circumradius=0.05,
    min_facet_angle=30,
    max_radius_surface_delaunay_ball=0.05,
    max_facet_distance=0.025,
    max_circumradius_edge_ratio=2.0,
    number_of_copies_in_output=4,
    outfile='periodic.mesh',
    # odt=True,
    # lloyd=True,
    verbose=False,
)

import pyvista as pv
pv.plot_itk(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [5]:
import meshio
meshio.write('multi_domain_periodic.msh', mesh, file_format='gmsh22', binary=False)

In [4]:
import meshio
meshio.write('multi_domain_periodic.msh', mesh)

WriteError: Specify entity information to deal with more than one cell type